In [4]:
import numpy as np
import random

class AntColonyOptimization:
    def __init__(self, node_coord, demand, num_trucks, truck_capacity, alpha=1, beta=2, evaporation_rate=0.5, iterations=100):
        self.node_coord = node_coord # Кординаты нодов
        self.demand = demand # Спрос в нодах
        self.num_trucks = num_trucks # Количество машин
        self.truck_capacity = truck_capacity # Вместительность машин
        self.alpha = alpha # Альфа-параметр (влияние феромонного следа)
        self.beta = beta # Бета-параметр (влияние эвристической информации)
        self.evaporation_rate = evaporation_rate # Скорость испарения феромонов
        self.iterations = iterations  # Количество итераций для проведения симуляции
        
        self.num_nodes = len(node_coord) # Общее количество узлов
        self.distance_matrix = self.calculate_distance_matrix() # Матрица расстояний между узлами
        self.pheromone_matrix = np.ones((self.num_nodes, self.num_nodes)) # Инициализация матрицы феромонов

    # Функция для вычисления матрицы расстояний на основе координат узлов
    def calculate_distance_matrix(self):
        distance_matrix = np.zeros((self.num_nodes, self.num_nodes))  # Инициализация матрицы расстояний
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if i != j:
                    distance_matrix[i][j] = np.linalg.norm(self.node_coord[i] - self.node_coord[j])  # Вычисление евклидова расстояния между узлами i и j
        return distance_matrix
    
    # Функция для выбора следующего узла
    def choose_next_node(self, current_node, visited_nodes):
        probabilities = [] # Список для хранения вероятностей перехода к каждому узлу
        for i in range(self.num_nodes):
            if i not in visited_nodes:  # Если узел не был посещен
                if self.distance_matrix[current_node][i] == 0:
                    eta = 0
                else:
                    # Вычисление значения эвристики (eta)
                    eta = (1.0 / self.distance_matrix[current_node][i]) ** self.beta
                # Вычисление значения феромона (tau)
                tau = self.pheromone_matrix[current_node][i] ** self.alpha
                probabilities.append(tau * eta)
            else: # Добавляем нулевую вероятность посещения
                probabilities.append(0)
         # Нормализация вероятностей
        sum_probabilities = np.sum(probabilities)
        if sum_probabilities == 0:
            return 0
        probabilities = probabilities / sum_probabilities
        # Выбор следующего узла на основе вероятностей
        return np.random.choice(range(self.num_nodes), p=probabilities)

    # Основная функция для запуска оптимизации муравьиной колонии
    def run(self):
        best_distance = float('inf') # Лучшее расстояние == бесконечность
        best_solution = None
    
        # Проходим по количеству итераций
        for iteration in range(self.iterations):
            solutions = []
            distances = []

            # Итерация для каждого муравья
            for ant in range(self.num_trucks):
                solution = [0]  # Устанавляваем точку старта на 0
                visited_nodes = {0} # Посещенные узлы
                current_load = 0 # Текущий груз
                
                # Цикл, пока не будут посещены все узлы
                while len(visited_nodes) < self.num_nodes:
                    current_node = solution[-1] # Текущий узел == последний добавленный
                    next_node = self.choose_next_node(current_node, visited_nodes) # Выбираем следующий узел
                    
                    # Если добавление следующего узла не превышает вместимость
                    if current_load + self.demand[next_node] <= self.truck_capacity:
                        current_load += self.demand[next_node]
                        solution.append(next_node)
                        visited_nodes.add(next_node)
                    # Возврат в депо
                    else: 
                        solution.append(0) 
                        current_load = 0

                solutions.append(solution) # Добавление решения в список решений
                distances.append(self.calculate_solution_distance(solution)) # Вычисление и добавление расстояния

            # Обновляем лучшее решение
            min_distance = min(distances)
            if min_distance < best_distance:
                best_distance = min_distance
                best_solution = solutions[distances.index(min_distance)]

            # Обновление уровней феромонов
            self.pheromone_matrix *= (1 - self.evaporation_rate)  # Применение изменений к матрице феромонов
            for solution, distance in zip(solutions, distances):
                for i in range(len(solution) - 1): # Повышение уровня феромонов вдоль путей, пройденных муравьями
                    self.pheromone_matrix[solution[i]][solution[i + 1]] += 1.0 / distance

        return best_solution, best_distance
    
    # Функция для вычисления общего расстояния до решения
    def calculate_solution_distance(self, solution):
        distance = 0
        for i in range(len(solution) - 1):
            # Добавление расстояния между последовательными узлами в решении
            distance += self.distance_matrix[solution[i]][solution[i + 1]]
        return distance

In [20]:
import os
import vrplib
import numpy as np
import random
import re

# Папка с vrp файлами
vrp_folder = './vrp/A/'

# Функция поиска всех файлов в папке
def find_txt_files(directory):
    txt_files = []
    sol_files = []
 
    for file in os.listdir(directory):
        if file.endswith(".vrp"):
            txt_files.append(file)
        if file.endswith(".sol"):
            sol_files.append(file)
            
    return txt_files, sol_files

# Функция для форматирования представления машрута
def format_route(path, demand):
    routes = []
    current_route = []
    current_demand = 0
    for node in path:
        if node == 0 and current_route:
            routes.append((current_route, current_demand))
            current_route = []
            current_demand = 0
        else:
            current_route.append(node)
            current_demand += demand[node]
    if current_route:
        routes.append((current_route, current_demand))

    return routes


vrp_files, sol_files = find_txt_files(vrp_folder)

In [21]:
average_difference = 0
# Цикл по фалам в папке
for i in range(len(vrp_files)):
    # Читаем файл vrplib
    vrp_problem = vrplib.read_instance(vrp_folder + vrp_files[i])
    # Парсим содержимое ключа comment
    numbers = re.findall(r'\d+', vrp_problem['comment'])
    numbers_list = [int(num) for num in numbers]
    # Задаем No_of_trucks и Optimal_value
    vrp_problem['No_of_trucks'] = numbers_list[-2]
    vrp_problem['Optimal_value'] = numbers_list[-1]
    # Проверяем чтобы из файла считались кординаты
    if 'node_coord' in vrp_problem.keys():
        # Задаем класс AntColonyOptimization
        a = AntColonyOptimization(vrp_problem['node_coord'], vrp_problem['demand'], vrp_problem['No_of_trucks'],  vrp_problem['capacity'], \
                                  alpha=1, beta=2, evaporation_rate=0.5, iterations=100)
        optimal_value = vrp_problem['Optimal_value']
        # Запускаем подсчет
        best_solution, best_distance = a.run()
        # Передаем получившийся пусть и best_distance в format_route
        path = best_solution
        cost = best_distance
        demand = vrp_problem['demand']
        # Создаем пути
        formatted_routes = format_route(path, demand)
        # Выводим пути
        output = ""
        for j, (route, route_demand) in enumerate(formatted_routes):
            output += f"Route #{j + 1}: {' '.join(map(str, route))} (Demand: {route_demand})\n"
        output += f"Cost {cost:.0f}"
        # Выводим информацию о VRP
        print(f'Name {vrp_files[i]}')
        print(f'Capacity {vrp_problem["capacity"]}')
        print(output)
        print('Optimal_value:', vrp_problem['Optimal_value'])
        print('Difference: {:.2f}%'.format((best_distance - optimal_value) / optimal_value * 100))
        average_difference += (best_distance - optimal_value) / optimal_value * 100
        print('*' * 40)
print(f'Average Difference {average_difference/len(vrp_files)}%')

Name A-n32-k5.vrp
Capacity 100
Route #1: 0 30 26 16 1 12 7 (Demand: 90)
Route #2: 27 24 14 6 2 3 23 18 (Demand: 95)
Route #3: 29 11 4 28 8 9 22 15 (Demand: 98)
Route #4: 13 17 19 31 21 (Demand: 80)
Route #5: 20 5 25 10 (Demand: 47)
Cost 857
Optimal_value: 784
Difference: 9.33%
****************************************
Name A-n33-k5.vrp
Capacity 100
Route #1: 0 22 23 28 18 31 11 (Demand: 90)
Route #2: 32 13 8 7 26 5 25 (Demand: 79)
Route #3: 2 20 4 12 10 30 (Demand: 93)
Route #4: 24 6 19 21 14 1 29 16 (Demand: 87)
Route #5: 15 27 17 9 3 (Demand: 97)
Cost 704
Optimal_value: 661
Difference: 6.50%
****************************************
Name A-n33-k6.vrp
Capacity 100
Route #1: 0 30 16 27 25 31 24 23 (Demand: 84)
Route #2: 28 32 10 14 17 (Demand: 77)
Route #3: 21 12 (Demand: 91)
Route #4: 1 6 18 7 19 5 (Demand: 92)
Route #5: 13 26 22 8 4 3 (Demand: 99)
Route #6: 11 29 15 20 2 9 (Demand: 98)
Cost 793
Optimal_value: 742
Difference: 6.93%
****************************************
Name A-n34-k5.